# Solution
## Step 1: Create dictionary
- Load the vocab file into numpy using readlines
## Step 2: Create paragraph vectors
- Load all the trainign and test data into an array. This will create 4x12500 length arrays.
 - train/pos
 - train/neg
 - test/pos
 - test/neg
 each of these arays in fact includes a paragraph
## Step 3: Cteate BOW
- tokenize each element in each array. This shanges shape of the vectors. Initially vector shapes are (12500, 1). 
- do wordcount (word, count)
- get rid of most common words
- 


In [39]:
# imports and env-variable
import numpy as mp
import os
import glob
from nltk.tokenize.moses import MosesTokenizer

BASE_DIR = '/home/ubuntu/CyrusProjects/mxnet-notebooks/python/tutorials'
PATH='/aclImdb'

In [100]:
with open(BASE_DIR + PATH + '/imdb.vocab') as f:
    dic = f.readlines()
dic= [x.strip() for x in dic]
dic = np.array(np.sort(dic))
dic = np.unique(dic)
dic = dic.tolist()
print(len(dic))

89527


In [94]:
arr = [1,2,3,4,5]
try:
    print(arr.index(3))
except ValueError:
    print("item does not exist")
    


2
[1, 2, 3, 4, 5]


In [40]:
# tokenizer.perl is from Moses: https://github.com/moses-smt/mosesdecoder/tree/master/scripts/tokenizer
def sentence_array_creator(path, root_dir):
    os.chdir(path)
    sentences = []
    for file in list(glob.glob("*.txt")):
        with open(file, 'r') as f:
            sentences.append(f.readline().strip().lower())
    return sentences
    os.chdir(root_dir)



In [44]:

os.chdir(BASE_DIR)
root_dir = os.getcwd()

base_dir = root_dir + path
pos_train_path = base_dir + '/train/pos'
neg_train_path = base_dir + '/train/neg'
pos_test_path = base_dir + '/test/pos'
neg_test_path = base_dir + '/test/neg'


pos_train_sentences = sentence_array_creator(pos_train_path, root_dir)
neg_train_sentences = sentence_array_creator(neg_train_path, root_dir)
pos_test_sentences = sentence_array_creator(pos_test_path, root_dir)
neg_test_sentences = sentence_array_creator(neg_test_path, root_dir)
print('{}; {}; {}; {}; '.format)

print("{}, {}, {}, {}".format(len(pos_train_sentences), len(neg_train_sentences), len(pos_test_sentences), len(neg_test_sentences)))


            
        


<built-in method format of str object at 0x7f5d2cc0ca98>
12500, 12500, 12500, 12500


In [117]:
def get_dic_index(word, dic):
    try:
        ret_val = dic.index(word)
    except ValueError: 
        ret_val = -1
    return ret_val

    
word = "aamir"
print(get_dic_index(word, dic))
    

101


In [135]:
def sentence_embedding(sentences):
    t = MosesTokenizer()
    bow = []
    for i in range(len(sentences)):
        elements = []
        tokens = t.tokenize(pos_test_sentences[i])
        for token in tokens:
            index = get_dic_index(token, dic)
            if index != -1:
                elements.append(index)
        bow.append(elements)
    return bow
    

    
            

In [ ]:
bow_pos_train_sentences = sentence_embedding(pos_train_sentences)


In [134]:
print(len(bow_pos_train_sentences))
print(bow_pos_train_sentences[0])

10
[23842, 4161, 39813, 64192, 58688, 10892, 29669, 35143, 52601, 17451, 68872, 2614, 4201, 28777, 79025, 61359, 39813, 49020, 20243, 88728, 1338, 9147, 9147, 79344, 27653, 68359, 48180, 71592, 5255, 38082, 87003, 39920, 1495, 55160, 69303, 39920, 1252, 88728, 43533, 39920, 75241, 88101, 9147, 9147, 79025, 51028, 39813, 31, 44960, 72491, 2614, 60829, 79025, 18644, 3728, 84937, 77941, 2614, 79158, 31, 77707, 34067, 24322, 10492, 79344, 39813, 31, 81915, 51028, 72991, 88923, 34482, 80083, 31383, 39920, 23842, 4161, 39813, 25712, 6862, 30648, 39813, 54998, 2614, 39920, 29879, 80083, 69287, 49020, 21992, 72991, 2614, 79405, 78534, 10892, 75241, 68354, 79025, 45010, 18491, 55925, 54754, 37329, 721, 33439, 80083, 81827, 31, 44574, 55160, 22853, 55234, 54754, 79073, 0, 9147, 9147, 31, 17007, 44960, 81915, 88193, 69303]


In [106]:
from nltk.tokenize.moses import MosesTokenizer, MosesDetokenizer


t =  MosesTokenizer()
tokens = t.tokenize(sentences[0])
print(tokens)
print('----------------------------------------')
print(np.unique(tokens))


['It', 'seems', 'Hal', 'Hartley', '&apos;s', 'films', 'are', 'kind', 'of', 'hit', 'or', 'miss', 'with', 'most', 'audiences', '.', 'This', 'film', 'will', 'be', 'no', 'exception', 'to', 'that', 'rule', '.', 'Fay', 'Grim', 'acts', 'as', 'a', 'sequel', 'to', 'Hartley', '&apos;s', '&apos;', 'Henry', 'Foole', '&apos;', 'from', '1998', '.', 'The', 'focus', 'this', 'time', 'is', 'on', 'Henry', '&apos;s', 'ex', 'wife', '(', 'played', 'to', 'perfection', 'by', 'the', 'always', 'welcome', 'Parker', 'Posey', ')', ',', 'who', 'is', 'being', 'pestered', 'by', 'CIA', 'goons', 'about', 'Henry', '&apos;s', 'unpublished', 'book', 'about', 'all', 'of', 'his', 'shady', 'dealings', '.', 'In', 'the', 'interim', 'of', 'all', 'of', 'this', ',', 'Fay', 'ends', 'up', 'on', 'an', 'odyssey', ',', 'dealing', 'with', 'international', 'spies', ',', 'etc', '.', 'The', 'film', 'does', 'get', 'a', 'bit', 'bogged', 'down', 'in', 'the', 'second', 'half', '.', 'If', 'you', '&apos;ve', 'been', 'a', 'fan', 'of', 'Hal', 'Ha

In [91]:
import os
import nltk
os.system('pip3 install nltk')
nltk.download('all')




[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/ubuntu/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

True

In [85]:
import urllib3
http = urllib3.PoolManager()
import os


url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
path = os.getcwd() + '/local_aclImdb_v1.tar.gz'
chunk_size = 1024

r = http.request('GET', url, preload_content=False)

with open(path, 'wb') as out:
    while True:
        data = r.read(chunk_size)
        if not data:
            break
        out.write(data)

r.release_conn()

In [86]:
import tarfile
tar = tarfile.open("aclImdb_v1.tar.gz")
tar.extractall()
tar.close()

In [55]:
with open('aclImdb/imdb.vocab') as vocab_file:
    vocab_array = vocab_file.readlines()
vocab_array = [x.strip() for x in vocab_array]
print(len(vocab_array))


89527


In [38]:
d = StringIO("M 21 72\nF 35 58")
np.loadtxt(d, dtype={'names': ('gender', 'age', 'weight'),'formats': ('S1', 'i4', 'f4')})

NameError: name 'StringIO' is not defined

In [ ]:
def build_dict(path):
    sentences = []
    currdir = os.getcwd()
    os.chdir('%s/pos/' % path)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences.append(f.readline().strip())
    os.chdir('%s/neg/' % path)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences.append(f.readline().strip())
    os.chdir(currdir)

    sentences = tokenize(sentences)

    print('Building dictionary..', end=' ')
    wordcount = dict()
    for ss in sentences:
        words = ss.strip().lower().split()
        for w in words:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1

    counts = wordcount.values()
    keys = wordcount.keys()

    sorted_idx = numpy.argsort(counts)[::-1]

    worddict = dict()

    for idx, ss in enumerate(sorted_idx):
        worddict[keys[ss]] = idx+2  # leave 0 and 1 (UNK)

    print(numpy.sum(counts), ' total words ', len(keys), ' unique words')

    return worddict

In [9]:
from nltk.tokenize.moses import MosesTokenizer, MosesDetokenizer


text = u'This, is a sentence with weird» symbols… appearing everywhere¿'
t, d = MosesTokenizer(), MosesDetokenizer()
sent = "This ain't funny. It's actually hillarious, yet double Ls. | [] < > [ ] & You're gonna shake it off? Don't?"
expected_tokens = [u'This', u'ain', u'&apos;t', u'funny', u'.', u'It', u'&apos;s', u'actually', u'hillarious', u',', u'yet', u'double', u'Ls', u'.', u'&#124;', u'&#91;', u'&#93;', u'&lt;', u'&gt;', u'&#91;', u'&#93;', u'&amp;', u'You', u'&apos;re', u'gonna', u'shake', u'it', u'off', u'?', u'Don', u'&apos;t', u'?']
expected_detokens = "This ain't funny. It's actually hillarious, yet double Ls. | [] < > [] & You're gonna shake it off? Don't?"
tokens = t.tokenize(sent)
print(tokens)
if tokens == expected_tokens:
    print("True")
detokens = d.detokenize(tokens)
if " ".join(detokens) == expected_detokens:
    print("True")

['This', 'ain', '&apos;t', 'funny', '.', 'It', '&apos;s', 'actually', 'hillarious', ',', 'yet', 'double', 'Ls', '.', '&#124;', '&#91;', '&#93;', '&lt;', '&gt;', '&#91;', '&#93;', '&amp;', 'You', '&apos;re', 'gonna', 'shake', 'it', 'off', '?', 'Don', '&apos;t', '?']
True
True


In [2]:
""" Referemce: http://deeplearning.net/tutorial/code/imdb_preprocess.py"""
"""
This script is what created the dataset pickled.

1) You need to download this file and put it in the same directory as this file.
https://github.com/moses-smt/mosesdecoder/raw/master/scripts/tokenizer/tokenizer.perl . Give it execution permission.

2) Get the dataset from http://ai.stanford.edu/~amaas/data/sentiment/ and extract it in the current directory.

3) Then run this script.
"""

dataset_path='aclImdb_v1.tar.gz'

import numpy
import _pickle as pkl

from collections import OrderedDict

import glob
import os

from subprocess import Popen, PIPE

# tokenizer.perl is from Moses: https://github.com/moses-smt/mosesdecoder/tree/master/scripts/tokenizer
tokenizer_cmd = ['./tokenizer.perl', '-l', 'en', '-q', '-']


def tokenize(sentences):

    print('Tokenizing..', end=' ')
    text = "\n".join(sentences)
    tokenizer = Popen(tokenizer_cmd, stdin=PIPE, stdout=PIPE)
    tok_text, _ = tokenizer.communicate(text)
    toks = tok_text.split('\n')[:-1]
    print('Done')

    return toks


def build_dict(path):
    sentences = []
    currdir = os.getcwd()
    os.chdir('%s/pos/' % path)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences.append(f.readline().strip())
    os.chdir('%s/neg/' % path)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences.append(f.readline().strip())
    os.chdir(currdir)

    sentences = tokenize(sentences)

    print('Building dictionary..', end=' ')
    wordcount = dict()
    for ss in sentences:
        words = ss.strip().lower().split()
        for w in words:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1

    counts = wordcount.values()
    keys = wordcount.keys()

    sorted_idx = numpy.argsort(counts)[::-1]

    worddict = dict()

    for idx, ss in enumerate(sorted_idx):
        worddict[keys[ss]] = idx+2  # leave 0 and 1 (UNK)

    print(numpy.sum(counts), ' total words ', len(keys), ' unique words')

    return worddict


def grab_data(path, dictionary):
    sentences = []
    currdir = os.getcwd()
    os.chdir(path)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences.append(f.readline().strip())
    os.chdir(currdir)
    sentences = tokenize(sentences)

    seqs = [None] * len(sentences)
    for idx, ss in enumerate(sentences):
        words = ss.strip().lower().split()
        seqs[idx] = [dictionary[w] if w in dictionary else 1 for w in words]

    return seqs


def main():
    # Get the dataset from http://ai.stanford.edu/~amaas/data/sentiment/
    path = dataset_path
    dictionary = build_dict(os.path.join(path, 'train'))

    train_x_pos = grab_data(path+'train/pos', dictionary)
    train_x_neg = grab_data(path+'train/neg', dictionary)
    train_x = train_x_pos + train_x_neg
    train_y = [1] * len(train_x_pos) + [0] * len(train_x_neg)

    test_x_pos = grab_data(path+'test/pos', dictionary)
    test_x_neg = grab_data(path+'test/neg', dictionary)
    test_x = test_x_pos + test_x_neg
    test_y = [1] * len(test_x_pos) + [0] * len(test_x_neg)

    f = open('imdb.pkl', 'wb')
    pkl.dump((train_x, train_y), f, -1)
    pkl.dump((test_x, test_y), f, -1)
    f.close()

    f = open('imdb.dict.pkl', 'wb')
    pkl.dump(dictionary, f, -1)
    f.close()

if __name__ == '__main__':
    main()


ImportError: No module named 'cPickle'